<a href="https://colab.research.google.com/github/Harmandeepkaur370/1st-/blob/main/1st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===========================
# 🫀 Heart Disease Patient Portal (Multi-use, Mobile-friendly, Colorful)
# Colab one-cell app: Login → Upload CSV → Train → Predict (Pie chart + History)
# ===========================
!pip install -q gradio scikit-learn pandas numpy matplotlib

import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time

# --------- Single-user credentials (edit as you like) ----------
USERNAME = "patient"
PASSWORD = "1234"

# --------- App state ----------
MODEL = None
TEST_ACC = None
FEATURE_COLS = [
    "age","sex","cp","trestbps","chol","fbs","restecg",
    "thalach","exang","oldpeak","slope","ca","thal"
]  # expected UCI heart features, order matters
HISTORY = []  # list of dicts

# --------- Pretty labels & tooltips ----------
LABELS = {
    "age": "🧑 Age (years)",
    "sex": "⚧ Sex (0=Female,1=Male)",
    "cp": "💢 Chest Pain Type (0–3)",
    "trestbps": "❤️ Resting BP (mmHg)",
    "chol": "🩸 Cholesterol (mg/dL)",
    "fbs": "🍬 Fasting Blood Sugar > 120 (0/1)",
    "restecg": "📈 Resting ECG (0–2)",
    "thalach": "🏃 Max Heart Rate",
    "exang": "🚴 Exercise Induced Angina (0/1)",
    "oldpeak": "📉 ST Depression",
    "slope": "⛰️ ST Slope (0–2)",
    "ca": "🧪 Major Vessels Colored (0–4)",
    "thal": "🧬 Thalassemia (1/2/3)"
}

HELP = {
    "cp": "0: Typical angina, 1: Atypical angina, 2: Non-anginal, 3: Asymptomatic",
    "restecg": "0: Normal, 1: ST-T abnormality, 2: LV hypertrophy",
    "slope": "0: Upsloping, 1: Flat, 2: Downsloping",
    "thal": "1: Normal, 2: Fixed defect, 3: Reversible defect"
}

# --------- Default UI ranges (safe generic defaults if CSV ranges unknown) ----------
RANGES = {
    "age": (18, 100, 45, 1),
    "sex": (0, 1, 1, 1),
    "cp": (0, 3, 0, 1),
    "trestbps": (80, 200, 120, 1),
    "chol": (100, 600, 220, 1),
    "fbs": (0, 1, 0, 1),
    "restecg": (0, 2, 1, 1),
    "thalach": (60, 220, 150, 1),
    "exang": (0, 1, 0, 1),
    "oldpeak": (0.0, 6.5, 1.0, 0.1),
    "slope": (0, 2, 1, 1),
    "ca": (0, 4, 0, 1),
    "thal": (1, 3, 2, 1)
}

# --------- Core functions ----------
def login_fn(user, pw):
    ok = (user == USERNAME and pw == PASSWORD)
    if ok:
        msg = (
            '<div class="banner ok"><div class="title">Welcome 👋</div>'
            f'<div class="sub">Hello <b>{user}</b> — upload your dataset to begin.</div></div>'
        )
        return msg, gr.update(visible=False), gr.update(visible=True), f"Welcome, {user}!"
    else:
        msg = ('<div class="banner risk"><div class="title">Invalid credentials</div>'
               '<div class="sub">Try username <b>patient</b>, password <b>1234</b> (or edit in code).</div></div>')
        return msg, gr.update(visible=True), gr.update(visible=False), "Welcome"

def train_fn(file):
    global MODEL, TEST_ACC, FEATURE_COLS, RANGES
    if file is None:
        raise gr.Error("Please upload a CSV first.")

    df = pd.read_csv(file.name)

    # Validate target & features
    if "target" not in df.columns:
        raise gr.Error("CSV must include a 'target' column (0 = No Disease, 1 = Disease).")
    missing = [c for c in FEATURE_COLS if c not in df.columns]
    if missing:
        raise gr.Error(f"Missing expected feature columns: {missing}")

    X = df[FEATURE_COLS]
    y = df["target"]

    # update slider ranges from data (more realistic UI)
    for c in FEATURE_COLS:
        cmin, cmax = float(X[c].min()), float(X[c].max())
        cmed = float(X[c].median())
        step = 0.1 if (X[c].dtype.kind in "fc" or c == "oldpeak") else 1
        pad = (cmax - cmin) * 0.05
        lo = round(cmin - pad, 2) if step == 0.1 else int(np.floor(cmin - pad))
        hi = round(cmax + pad, 2) if step == 0.1 else int(np.ceil(cmax + pad))
        default = round(cmed, 2) if step == 0.1 else int(round(cmed))
        # keep within sensible global defaults too
        glo_lo, glo_hi, glo_def, glo_step = RANGES[c]
        lo = max(lo, glo_lo); hi = min(hi, glo_hi)
        default = min(max(default, lo), hi)
        RANGES[c] = (lo, hi, default, step)

    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    MODEL = LogisticRegression(max_iter=1000)
    MODEL.fit(X_tr, y_tr)
    TEST_ACC = float(accuracy_score(y_te, MODEL.predict(X_te)))

    info = (f'<div class="banner ok"><div class="title">Model Trained ✅</div>'
            f'<div class="sub">Test Accuracy: <b>{TEST_ACC:.2f}</b>. Fill the patient form to predict.</div></div>')
    return info, gr.update(visible=True), gr.update(visible=True)

def predict_fn(age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal, username):
    if MODEL is None:
        raise gr.Error("Please train the model first.")
    values = [age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal]
    X_in = pd.DataFrame([values], columns=FEATURE_COLS)
    pred = int(MODEL.predict(X_in)[0])
    probs = MODEL.predict_proba(X_in)[0]
    p_no = float(probs[0] * 100)
    p_yes = float(probs[1] * 100)

    # result banner
    if pred == 0:
        banner = f"""
        <div class="banner ok">
            <div class="title">✅ No Heart Disease</div>
            <div class="sub">Estimated confidence (No Disease): {p_no:.2f}%</div>
        </div>"""
    else:
        banner = f"""
        <div class="banner risk">
            <div class="title">❗ Heart Disease Likely</div>
            <div class="sub">Estimated confidence (Disease): {p_yes:.2f}%</div>
        </div>"""

    # pie (matplotlib)
    labels = ["No Disease", "Disease"]
    fig, ax = plt.subplots(figsize=(4,4))
    ax.pie([p_no, p_yes], labels=labels, autopct="%1.1f%%", startangle=90,
           colors=["#4FC3F7", "#FF6F91"], wedgeprops={"linewidth":1,"edgecolor":"white"})
    ax.axis("equal")
    ax.set_title("Prediction Probability")

    # history
    rec = {
        "time": time.strftime("%Y-%m-%d %H:%M:%S"),
        "user": username if username else USERNAME,
        "prediction": "No Disease" if pred == 0 else "Disease",
        "prob_no(%)": round(p_no, 2),
        "prob_yes(%)": round(p_yes, 2),
        "age": age, "sex": sex, "cp": cp, "trestbps": trestbps, "chol": chol,
        "fbs": fbs, "restecg": restecg, "thalach": thalach, "exang": exang,
        "oldpeak": oldpeak, "slope": slope, "ca": ca, "thal": thal
    }
    HISTORY.append(rec)
    df_hist = pd.DataFrame(HISTORY)[["time","user","prediction","prob_no(%)","prob_yes(%)"] + FEATURE_COLS]
    df_hist = df_hist.iloc[::-1].reset_index(drop=True)

    return banner, f"{p_no:.2f}%", f"{p_yes:.2f}%", fig, df_hist

def reset_form_fn():
    # return defaults from RANGES for each slider/dropdown
    outs = []
    for c in FEATURE_COLS:
        _, _, default, _ = RANGES[c]
        outs.append(default)
    return outs

def clear_history_fn():
    HISTORY.clear()
    return pd.DataFrame(columns=["time","user","prediction","prob_no(%)","prob_yes(%)"] + FEATURE_COLS)

# --------- UI (Gradio Blocks) ----------
CSS = """
:root { --bg1:#f0f9ff; --bg2:#e0f2f1; --card:#ffffff; --ok:#e6fffa; --risk:#fff5f5; --text:#0f172a; }
.gradio-container { font-family: Inter, ui-sans-serif, system-ui; }
.app-bg {
  background: radial-gradient(900px 500px at 5% 10%, #e3f2fd 0, transparent 50%),
              radial-gradient(800px 500px at 95% 15%, #e0f7fa 0, transparent 45%),
              linear-gradient(180deg, var(--bg1), var(--bg2));
  min-height: 100vh; padding: 12px;
}
@media (max-width: 640px) { .card { padding: 12px; } .header { font-size: 20px; } }
.card {
  background: var(--card); border-radius: 16px; padding: 18px; box-shadow: 0 8px 24px rgba(0,0,0,0.06);
}
.header {
  font-size: 26px; font-weight: 800; color: var(--text); margin-bottom: 6px;
}
.subtle { color: #334155; }
.banner { border-radius: 14px; padding: 14px 16px; margin: 6px 0 4px; }
.banner.ok { background: linear-gradient(90deg, #e6fffa, #f0fff4); border: 1px solid #9ae6b4; }
.banner.risk { background: linear-gradient(90deg, #fff5f5, #fff0f1); border: 1px solid #feb2b2; }
.banner .title { font-size: 18px; font-weight: 700; }
.banner .sub { font-size: 13px; color: #334155; }
.section-title { font-weight: 700; font-size: 16px; margin: 8px 0 6px; }
.footer-note { font-size: 12px; color:#475569; }
th, td { font-size: 12px; }
"""

with gr.Blocks(css=CSS, theme=gr.themes.Soft()) as demo:
    gr.HTML('<div class="app-bg">')
    # Header
    with gr.Row():
        with gr.Column(scale=1):
            welcome = gr.Markdown("Welcome")
            gr.HTML('<div class="header">🏥 Heart Disease Patient Portal</div>'
                    '<div class="subtle">Multi-use • Mobile-friendly • Educational (not medical advice)</div>')

    # LOGIN
    with gr.Row():
        with gr.Column(scale=1):
            login_card = gr.Group(visible=True)
            with login_card:
                gr.HTML('<div class="card"><div class="section-title">🔐 Login</div>'
                        '<div class="subtle">Use credentials to continue. (Default: patient / 1234)</div>')
                user_in = gr.Textbox(label="👤 Username", placeholder="patient")
                pass_in = gr.Textbox(label="🔑 Password", type="password", placeholder="1234")
                login_btn = gr.Button("Login", variant="primary")
                login_msg = gr.HTML("")
                gr.HTML('</div>')

    # DATA / TRAIN
    data_card = gr.Group(visible=False)
    with data_card:
        with gr.Row():
            with gr.Column():
                gr.HTML('<div class="card"><div class="section-title">📦 Upload CSV & Train</div>'
                        '<div class="subtle">Required columns: '
                        '<code>age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal, target</code>.</div>')
                csv_file = gr.File(label="CSV file", file_types=[".csv"])
                train_btn = gr.Button("Train Model", variant="primary")
                train_info = gr.HTML("")
                gr.HTML('</div>')

    # PREDICT
    predict_card = gr.Group(visible=False)
    with predict_card:
        with gr.Row():
            # Left: form (mobile stacks neatly)
            with gr.Column(scale=2):
                gr.HTML('<div class="card"><div class="section-title">🧾 Patient Form</div>')
                # Build inputs from RANGES (will update after training)
                comp_map = {}
                for c in FEATURE_COLS:
                    lo, hi, default, step = RANGES[c]
                    if c in ["sex","fbs","exang"]:
                        comp = gr.Dropdown(choices=[0,1], value=default, label=LABELS[c])
                    elif c in ["cp","restecg","slope"]:
                        maxv = int(hi); minv = int(lo)
                        comp = gr.Dropdown(choices=list(range(minv, maxv+1)), value=int(default),
                                           label=f"{LABELS[c]}  ▸ {HELP.get(c,'')}")
                    elif c in ["ca"]:
                        comp = gr.Dropdown(choices=[0,1,2,3,4], value=int(default), label=LABELS[c])
                    elif c in ["thal"]:
                        comp = gr.Dropdown(choices=[1,2,3], value=int(default),
                                           label=f"{LABELS[c]}  ▸ {HELP.get(c,'')}")
                    else:
                        comp = gr.Slider(minimum=lo, maximum=hi, value=default, step=step, label=LABELS[c])
                    comp_map[c] = comp

                reset_btn = gr.Button("↩️ Reset to Defaults")

                gr.HTML('<div class="footer-note">Note: This tool is for learning only and not a substitute for medical advice.</div></div>')

            # Right: results
            with gr.Column(scale=1):
                gr.HTML('<div class="card"><div class="section-title">📊 Diagnosis</div>')
                banner_html = gr.HTML("")
                prob_no_lbl = gr.Label(label="Probability (No Disease)")
                prob_yes_lbl = gr.Label(label="Probability (Disease)")
                pie_plot = gr.Plot()
                predict_btn = gr.Button("🔍 Predict Risk", variant="primary")
                gr.HTML('</div>')

    # HISTORY
    history_card = gr.Group(visible=False)
    with history_card:
        with gr.Row():
            with gr.Column():
                gr.HTML('<div class="card"><div class="section-title">🗂️ Prediction History</div>')
                history_tbl = gr.Dataframe(headers=["time","user","prediction","prob_no(%)","prob_yes(%)"] + FEATURE_COLS, value=[])
                clear_btn = gr.Button("🧹 Clear History")
                gr.HTML('</div>')

    gr.HTML('</div>')  # end bg

    # --- Wire events ---
    login_btn.click(login_fn, inputs=[user_in, pass_in], outputs=[login_msg, login_card, data_card, welcome])
    train_btn.click(train_fn, inputs=[csv_file], outputs=[train_info, predict_card, history_card])

    # Predict button inputs (in exact order)
    predict_inputs = [comp_map[c] for c in FEATURE_COLS] + [user_in]
    predict_btn.click(predict_fn, inputs=predict_inputs,
                      outputs=[banner_html, prob_no_lbl, prob_yes_lbl, pie_plot, history_tbl])

    # Reset form to defaults
    reset_btn.click(reset_form_fn, inputs=[], outputs=[comp_map[c] for c in FEATURE_COLS])

    # Clear history
    clear_btn.click(clear_history_fn, inputs=[], outputs=[history_tbl])

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e1ce9accbef4ad275b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
